# Paquetes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

# Parametros

In [16]:
# MODEL

_LANGUAGE_         = 'es'
_PRETRAINED_LM_    = 'dccuchile/bert-base-spanish-wwm-cased'
_PREPROCESS_TEXT_  = False
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = None
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'feature_extractor_checkPoints_en'
_LOGGING_STEPS_    = 50


# TRAIN

_NO_GPUS_          = 1
_BATCH_SIZE_       = 100
_EPOCHS_           = {'gender': 5, 'variety': 5}
_LEARNING_RATE_    = 1e-4

# Dataset

In [3]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'great britain' : 2,
                  'ireland'      : 3,
                  'new zealand'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [4]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [5]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
    
vocab = tokenizer.get_vocab()

In [6]:
# CREATE ONE INSTANCE PER DATA SPLIT

from DatasetPAN17 import BasePAN17, DatasetPAN17

baseTrain = BasePAN17(Dir              = 'data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir              = 'data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Tokenizing...
    Done

Total Instances: 84000


Reading data...
    Done
Tokenizing...
    Done

Total Instances: 56000



In [7]:
# CREATE DATA LOADER FOR EVERY TASK

tasks = ['gender', 'variety']

dataset_dict = {}
for task in tasks:
    dataset_dict[task] = DatasetPAN17(Base_Dataset = baseTrain, label = task)

# Model

In [17]:
from transformers import AutoAdapterModel


model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertAdapterModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertAdapterModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initial

# Heads

In [18]:
num_v           = len(baseTrain.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}

# Add classification heads
for task_name in tasks:
    
    model.add_classification_head(
        head_name    = task_name,
        num_labels   = num_labels_dict[task_name],
      )

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Training

In [19]:
from Training import train_model_with_heads

train_model_with_heads(model         = model,
                       dataset_dict  = dataset_dict,
                       epochs        = _EPOCHS_,
                       batch_size    = _BATCH_SIZE_,
                       no_gpus       = _NO_GPUS_,
                       output_dir    = _OUTPUT_DIR_, 
                       logging_steps = _LOGGING_STEPS_,
                       learning_rate = _LEARNING_RATE_)

/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 84000
  Num Epochs = 5
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 4200


Step,Training Loss
50,0.649200
100,0.630400
150,0.636800
200,0.638800
250,0.632000
300,0.625700
350,0.623000
400,0.624400
450,0.622800
500,0.624600


Saving model checkpoint to feature_extractor_checkPoints_en/gender/checkpoint-500
Configuration saved in feature_extractor_checkPoints_en/gender/checkpoint-500/config.json
Model weights saved in feature_extractor_checkPoints_en/gender/checkpoint-500/pytorch_model.bin
Saving model checkpoint to feature_extractor_checkPoints_en/gender/checkpoint-1000
Configuration saved in feature_extractor_checkPoints_en/gender/checkpoint-1000/config.json
Model weights saved in feature_extractor_checkPoints_en/gender/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to feature_extractor_checkPoints_en/gender/checkpoint-1500
Configuration saved in feature_extractor_checkPoints_en/gender/checkpoint-1500/config.json
Model weights saved in feature_extractor_checkPoints_en/gender/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to feature_extractor_checkPoints_en/gender/checkpoint-2000
Configuration saved in feature_extractor_checkPoints_en/gender/checkpoint-2000/config.json
Model weights sa

Step,Training Loss
50,1.884000
100,1.771300
150,1.693200
200,1.652000
250,1.603600
300,1.589600
350,1.558100
400,1.553200
450,1.539500
500,1.545300


Saving model checkpoint to feature_extractor_checkPoints_en/variety/checkpoint-500
Configuration saved in feature_extractor_checkPoints_en/variety/checkpoint-500/config.json
Model weights saved in feature_extractor_checkPoints_en/variety/checkpoint-500/pytorch_model.bin
Saving model checkpoint to feature_extractor_checkPoints_en/variety/checkpoint-1000
Configuration saved in feature_extractor_checkPoints_en/variety/checkpoint-1000/config.json
Model weights saved in feature_extractor_checkPoints_en/variety/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to feature_extractor_checkPoints_en/variety/checkpoint-1500
Configuration saved in feature_extractor_checkPoints_en/variety/checkpoint-1500/config.json
Model weights saved in feature_extractor_checkPoints_en/variety/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to feature_extractor_checkPoints_en/variety/checkpoint-2000
Configuration saved in feature_extractor_checkPoints_en/variety/checkpoint-2000/config.json
Model

# Testing

In [20]:
from TestingPAN17 import test_model_with_heads

accuracy = test_model_with_heads(model, baseTest)


  3%|█▍                                       | 96/2800 [00:19<08:37,  5.23it/s]


  7%|██▊                                     | 193/2800 [00:37<08:03,  5.39it/s]


 10%|████▏                                   | 290/2800 [00:55<07:49,  5.35it/s]


 14%|█████▌                                  | 387/2800 [01:13<07:23,  5.44it/s]


 17%|██████▉                                 | 484/2800 [01:31<07:19,  5.27it/s]


...

 87%|█████████████████████████████████▊     | 2424/2800 [07:40<01:11,  5.28it/s]


 90%|███████████████████████████████████    | 2521/2800 [07:58<00:51,  5.44it/s]


 94%|████████████████████████████████████▍  | 2618/2800 [08:16<00:33,  5.43it/s]


 97%|█████████████████████████████████████▊ | 2715/2800 [08:35<00:16,  5.29it/s]


100%|███████████████████████████████████████| 2800/2800 [08:50<00:00,  5.27it/s]


In [21]:
accuracy

{'gender': 0.7232142857142857,
 'variety': 0.7164285714285714,
 'joint': 0.5135714285714286}